In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [3]:
## preprocess the data
# drop irrelevant columns


### ye mene ek baar run kar di thi aur phir mene phirse run kiya to ye error aaya ki column nahi mila
### isliye mene is line ko comment kar diya hai taaki aapko samajh aaye ki mene kya kiya tha



data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


In [4]:
data

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              1   
9998          772   Germany    Male   42       3   75075.31              2   
9999          792    France  Female   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0             1               1        101348.88       1  
1             0               1        112542.58       0  
2             1               0        113931.57       1  
3             0               0         93826.63       0  
4             1               1         79084.10       0  
...         ...             ...              ...     ...  
9995          1               0         96270.64       0  
9996          1               1        101699.77       0  
9997          0               1         42085.58       1  
9998          1               0         92888.52       1  
9999          1               0         38190.78       0  

[10000 rows x 11 columns]

In [5]:
# encode categorical columns

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France       0   42       2       0.00              1   
1             608     Spain       0   41       1   83807.86              1   
2             502    France       0   42       8  159660.80              3   
3             699    France       0   39       1       0.00              2   
4             850     Spain       0   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France       1   39       5       0.00              2   
9996          516    France       1   35      10   57369.61              1   
9997          709    France       0   36       7       0.00              1   
9998          772   Germany       1   42       3   75075.31              2   
9999          792    France       0   28       4  130142.79              1   

      HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0             1               1        101348.88       1  
1             0               1        112542.58       0  
2             1               0        113931.57       1  
3             0               0         93826.63       0  
4             1               1         79084.10       0  
...         ...             ...              ...     ...  
9995          1               0         96270.64       0  
9996          1               1        101699.77       0  
9997          0               1         42085.58       1  
9998          1               0         92888.52       1  
9999          1               0         38190.78       0  

[10000 rows x 11 columns]

In [6]:
## one hot encode 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [7]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [8]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [10]:
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [11]:
## COMBINE ALL ONE HOT ENCODED COLUMNS WITH ORIGINAL DATA
data= pd.concat([data.drop(['Geography'], axis=1), geo_encoded_df], axis=1)
data.head()

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2       0.00              1          1   
1          608       0   41       1   83807.86              1          0   
2          502       0   42       8  159660.80              3          1   
3          699       0   39       1       0.00              2          0   
4          850       0   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0               1        101348.88       1               1.0   
1               1        112542.58       0               0.0   
2               0        113931.57       1               1.0   
3               0         93826.63       0               1.0   
4               1         79084.10       0               0.0   

   Geography_Germany  Geography_Spain  
0                0.0              0.0  
1                0.0              1.0  
2                0.0              0.0  
3                0.0              0.0  
4                0.0              1.0

In [12]:
## save the encoders and scaler
with open('label_encoder_gender.pkl' , 'wb') as file:
    pickle.dump(label_encoder_gender , file)

with open('onehot_encoder_geo.pkl' , 'wb') as file:
    pickle.dump(onehot_encoder_geo , file)


In [13]:
# divide the dataset into independent and dependent features
X = data.drop('Exited', axis=1)
y = data['Exited']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
X_train

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
9254          686       1   32       6       0.00              2          1   
1561          632       1   42       4  119624.60              2          1   
1670          559       1   24       3  114739.92              1          1   
6087          561       0   27       9  135637.00              1          1   
6669          517       1   56       9  142147.32              1          0   
...           ...     ...  ...     ...        ...            ...        ...   
5734          768       1   54       8   69712.74              1          1   
5191          682       0   58       1       0.00              1          1   
5390          735       0   38       1       0.00              3          0   
860           667       1   43       8  190227.46              1          1   
7270          697       1   51       1  147910.30              1          1   

      IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
9254               1        179093.26               1.0                0.0   
1561               1        195978.86               0.0                1.0   
1670               0         85891.02               0.0                0.0   
6087               0        153080.40               1.0                0.0   
6669               0         39488.04               1.0                0.0   
...              ...              ...               ...                ...   
5734               1         69381.05               1.0                0.0   
5191               1           706.50               1.0                0.0   
5390               0         92220.12               1.0                0.0   
860                0         97508.04               1.0                0.0   
7270               1         53581.14               0.0                1.0   

      Geography_Spain  
9254              0.0  
1561              0.0  
1670              1.0  
6087              0.0  
6669              0.0  
...               ...  
5734              0.0  
5191              0.0  
5390              0.0  
860               0.0  
7270              0.0  

[8000 rows x 12 columns]

In [15]:
## scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [17]:
with open ('scaler.pkl' , 'wb') as file:
    pickle.dump(scaler , file)

ANN Implementation
------

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime

In [19]:
model = Sequential([
    Dense(64 , activation='relu' , input_shape=(X_train.shape[1] , )),  # H:! Connected with input layer
    Dense(32 , activation='relu' ),   # HL2
    Dense(1 , activation='sigmoid' ),  # output layer
])

c:\Users\nikhi\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [22]:
# compile the model
model.compile(optimizer=opt ,
              loss='binary_crossentropy' ,
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard

## set up the tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
## set up early stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5 , restore_best_weights=True)

In [25]:
## Training the model

history = model.fit(X_train , y_train ,validation_data=(X_test , y_test) ,epochs=100,
                    callbacks =[tensorflow_callback , early_stopping_callback] )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8356 - loss: 0.3968 - val_accuracy: 0.8540 - val_loss: 0.3572
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8570 - loss: 0.3545 - val_accuracy: 0.8605 - val_loss: 0.3536
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8572 - loss: 0.3487 - val_accuracy: 0.8595 - val_loss: 0.3441
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8608 - loss: 0.3431 - val_accuracy: 0.8525 - val_loss: 0.3436
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8584 - loss: 0.3413 - val_accuracy: 0.8655 - val_loss: 0.3542
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8620 - loss: 0.3412 - val_accuracy: 0.8595 - val_loss: 0.3467
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8635 - loss: 0.3353 - val_accuracy: 0.8580 - val_loss: 0.3647
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8621 - loss: 0.3329 - val_accu

In [26]:
model.save("model.h5")

In [27]:
# load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 55208), started 0:00:22 ago. (Use '!kill 55208' to kill it.)

In [ ]:
## load the picjle file

